# Full MNIST dataset project

Setup for fastai and PyTorch

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

     |████████████████████████████████| 720 kB 5.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 46 kB 3.8 MB/s 
     |████████████████████████████████| 186 kB 37.1 MB/s 
     |████████████████████████████████| 56 kB 3.7 MB/s 
     |████████████████████████████████| 51 kB 302 kB/s 
Mounted at /content/gdrive


First, we download MNIST dataset using fastai helper function

In [2]:
path = untar_data(URLs.MNIST)

In [3]:
path.ls()

(#2) [Path('/root/.fastai/data/mnist_png/training'),Path('/root/.fastai/data/mnist_png/testing')]

In [4]:
training = (path/'training').ls().sorted()
testing = (path/'testing').ls().sorted()

Setting image size and batch size

In [5]:
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
BATCH_SIZE = 256

A simple function for creating a dataset given fastai class L with all directories that contain images

In [6]:
def create_dset(dir_list):
  x_list = []
  y_list = []
  n = 0
  for num in dir_list:
    num = num.ls().sorted()
    x_tensor = torch.stack([tensor(Image.open(o)) for o in num]).float()/255
    y_tensor = tensor([n] * len(x_tensor)).unsqueeze(1)
    n += 1
    x_list.append(x_tensor)
    y_list.append(y_tensor)
  x = torch.cat(x_list).view(-1, IMAGE_WIDTH*IMAGE_HEIGHT)
  y = torch.cat(y_list,0).view(-1)
  dset = list(zip(x,y))
  return dset

Create dataset and validation dataset and check their sizes

In [7]:
dset = create_dset(training)

In [8]:
len(dset)

60000

In [9]:
valid_dset = create_dset(testing)

In [10]:
len(valid_dset)

10000

Now, we need to define loss function. We can use Cross Entropy Loss as it is useful for multi-classification problems.

In [11]:
mnist_loss = nn.CrossEntropyLoss()

Next, let's create a simple linear neural network with ReLU activation and 30 hidden units.

In [12]:
simple_net = nn.Sequential(
    nn.Linear(IMAGE_WIDTH*IMAGE_HEIGHT,30),
    nn.ReLU(),
    nn.Linear(30,10)
)

We also need to create DataLoaders from our datasets

In [13]:
train_dl = DataLoader(dset, batch_size=BATCH_SIZE)
valid_dl = DataLoader(valid_dset, batch_size=BATCH_SIZE)
dls = DataLoaders(train_dl, valid_dl)

In [14]:
xb, yb = first(train_dl)
xb.shape, yb.shape

(torch.Size([256, 784]), torch.Size([256]))

And a validation accuracy function

In [15]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    m,idx = torch.max(preds,1)
    correct = (idx==yb)
    return correct.float().mean()

In [16]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb), yb) for xb,yb in valid_dl]
    return round(torch.stack(accs).mean().item(), 4)


Now, we are ready to create a basic optimiser and training loop

In [17]:
class BasicOptim:
    def __init__(self,params,lr): self.params,self.lr = list(params),lr

    def step(self, *args, **kwargs):
        for p in self.params: p.data -= p.grad.data * self.lr

    def zero_grad(self, *args, **kwargs):
        for p in self.params: p.grad = None

In [18]:
opt = BasicOptim(simple_net.parameters(),lr=0.01)

In [19]:
def calc_grad(xb, yb, model):
    preds = model(xb)
    loss = mnist_loss(preds, yb)
    loss.backward()

In [20]:
def train_epoch(model):
    for xb,yb in train_dl:
      calc_grad(xb, yb, model)
      opt.step()
      opt.zero_grad()

In [21]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoch(model), end=' ')

Training for 10 epochs

In [22]:
train_model(simple_net, 10)

0.122 0.122 0.1734 0.2807 0.3534 0.3913 0.419 0.4388 0.4563 0.4743 

We see that it works! So, now let's create a learner to have a nicer output

In [23]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

In [24]:
learn.fit(10,0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,0.334885,1.459463,0.476500,00:01
1,0.325761,1.405799,0.490900,00:01
2,0.317931,1.357240,0.505300,00:01
3,0.311232,1.312934,0.516700,00:01
4,0.305470,1.272485,0.528000,00:01
5,0.300536,1.235247,0.539200,00:01
6,0.296287,1.200743,0.551100,00:01
7,0.292509,1.169590,0.560900,00:01
8,0.289095,1.140808,0.569800,00:01
9,0.286168,1.114606,0.578900,00:01


Now, let's increase a number of hidden units in our NN and train our model for longer.

In [25]:
bigger_simple_net = nn.Sequential(
    nn.Linear(IMAGE_WIDTH*IMAGE_HEIGHT,300),
    nn.ReLU(),
    nn.Linear(300,10)
)

In [26]:
learn = Learner(dls, bigger_simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

Fitting for 700 epochs - 97.78 % accuracy on validation dataset

In [27]:
learn.fit(700,0.01)

epoch,train_loss,valid_loss,batch_accuracy,time
0,1.503227,2.741359,0.100900,00:02
1,0.919128,2.609979,0.108200,00:02
2,0.654945,2.227224,0.237000,00:02
3,0.531045,1.973279,0.324400,00:02
4,0.464120,1.800685,0.370900,00:02
5,0.423006,1.670705,0.403400,00:02
6,0.395025,1.566189,0.432400,00:02
7,0.374441,1.478840,0.455100,00:02
8,0.358588,1.403574,0.476200,00:02
9,0.345864,1.338011,0.498200,00:02


Let's create a function for predicting one single digit from the dataset and test a couple of different ones to make sure that our model is actually working

In [28]:
def predict_single_digit(model, path_to_image):
  image = Image.open(path_to_image)
  digit = tensor(image).float()/255
  digit = digit.view(-1)
  preds = model(digit)
  m, idx = torch.max(preds,0)
  return idx

In [29]:
nums = (path/'testing'/'6').ls().sorted()
image = Image.open(nums[0])
image

In [30]:
predict_single_digit(bigger_simple_net, nums[0])

tensor(6)

In [31]:
nums = (path/'testing'/'0').ls().sorted()
image = Image.open(nums[0])
image

In [32]:
predict_single_digit(bigger_simple_net, nums[0])

tensor(0)

In [33]:
nums = (path/'testing'/'9').ls().sorted()
image = Image.open(nums[0])
image

In [34]:
predict_single_digit(bigger_simple_net, nums[0])

tensor(9)

Save model in ONNX format and check that it works right

In [35]:
x = torch.randn(784)
torch_out = bigger_simple_net(x)

In [36]:
torch.onnx.export(bigger_simple_net,x,"mnist_nn.onnx",export_params=True)

In [ ]:
!pip install onnx
!pip install onnxruntime
import onnxruntime
import onnx

onnx_model = onnx.load("mnist_nn.onnx")
onnx.checker.check_model(onnx_model)

In [40]:
ort_session = onnxruntime.InferenceSession("mnist_nn.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


In [42]:
image = Image.open(nums[0])
digit = tensor(image).float()/255
digit = digit.view(-1)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(digit)}
ort_outs = ort_session.run(None, ort_inputs)
image

In [43]:
np.argmax(ort_outs)

9

Also, save weights and check that everything works

In [47]:
torch.save(bigger_simple_net, 'mnist.pth')

In [48]:
# Definition of the NN class must be present
model = torch.load('mnist.pth')

In [49]:
# Digit is 9
preds = model(digit)
m,i = torch.max(preds,0)
i

tensor(9)

In both cases, saved models work as intented.